In [3]:
import pandas as pd
import requests
from datetime import date
from sqlalchemy import create_engine, Table, MetaData, select
from time import sleep

# Receita para baixar todas as editoras

Este notebook fará uma conexão com um arquivo sqlite previamente criado na mesma pasta, executa uma série de requisições para o site da Feira do Livro para baixar a lista de editoras, compila as informações e grava no sqlite.

## Conexão ao SQLite

In [4]:
db = create_engine('sqlite:///livros_usp_2024.db')

## Baixar todos as editoras do site da Feira do Livro

In [5]:
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    # 'cookie': 'PHPSESSID=ha1p3kg5ik1njdnk3iekdfqcvd',
    'priority': 'u=1, i',
    'referer': 'https://festadolivro.edusp.com.br/26-festa-do-livro-da-usp/editoras',
    'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
}


In [21]:
params = {
    'flag': '26-festa-do-livro-da-usp',
    'page': 0
}


In [22]:
resposta = requests.get('https://festadolivro.edusp.com.br/api/v1/event-publishers', params=params, headers=headers)

In [ ]:
if resposta.status_code == 200:
    print('Sucesso')

In [24]:
dados = resposta.json()['data']

In [25]:
resultados_df = pd.DataFrame.from_dict(resposta.json()['data'])

In [26]:
resultados_df['data_atualizacao'] = pd.to_datetime(date.today())


In [ ]:
print('Livros encontrados: ' + str(len(resultados_df)))


In [28]:
resultados_df = resultados_df.drop(['address', 'social'], axis=1)

In [ ]:
resultados_df

In [30]:
# with db.connect() as conn:
#     resultados_df.to_sql('editoras_usp', conn, if_exists='append', index=False)

## Buscar livros já cadastrados

In [31]:
editoras_usp_tbl = Table('editoras_usp', MetaData(), autoload_with=db)


In [32]:
select_query = select(editoras_usp_tbl).order_by(editoras_usp_tbl.c.url)


In [33]:
with db.connect() as conn:
    editoras_usp = pd.read_sql(select_query, conn)


In [ ]:
# Quantas editoras já estão no BD?
len(editoras_usp) 

In [ ]:
while 0 < 30:
    params['page'] = params['page'] + 1
    resposta = requests.get('https://festadolivro.edusp.com.br/api/v1/event-publishers', params=params, headers=headers)
    if resposta.status_code == 200:
        resultados_df = pd.DataFrame.from_dict(resposta.json()['data'])
        resultados_df['data_atualizacao'] = pd.to_datetime(date.today())
        print('Editoras encontradas: ' + str(len(resultados_df)))
        if len(resultados_df) > 0:
            resultados_df = resultados_df.drop(['address', 'social'], axis=1)
            with db.connect() as conn:
                editoras_usp = pd.read_sql(select_query, conn)
            resultados_df = resultados_df.loc[~resultados_df['url'].isin(editoras_usp['url']),].copy()
            if len(resultados_df) > 0:
                print('Editoras novas para gravar: ' + str(len(resultados_df)))
                with db.connect() as conn:
                    resultados_df.to_sql('editoras_usp', conn, if_exists='append', index=False)
            else:
                print('Sem editoras novas para gravar')
        else:
            break
    else:
        print('Erro na página' + str(params['page']))
    sleep(5)
    print(params['page'])